In [4]:
import pandas as pd
import numpy as np
from helpers.data_load import load_data_from_txt

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

import warnings
warnings.filterwarnings('ignore')



In [5]:
import time
import datetime

In [2]:
data = load_data_from_txt(path='data/data.txt')

In [12]:
data

,EURUSD,numdate,time,open,max,min,close,vol
EURUSD,5,20200701,500,1.12332,1.12364,1.12310,1.12330,245
EURUSD,5,20200701,1000,1.12327,1.12356,1.12310,1.12330,1414
EURUSD,5,20200701,1500,1.12331,1.12351,1.12310,1.12322,1317
EURUSD,5,20200701,2000,1.12322,1.12340,1.12300,1.12323,992
EURUSD,5,20200701,2500,1.12321,1.12346,1.12297,1.12314,982
...,...,...,...,...,...,...,...,...
EURUSD,5,20200930,234500,1.17256,1.17275,1.17230,1.17249,2480
EURUSD,5,20200930,235000,1.17241,1.17258,1.17220,1.17245,2192
EURUSD,5,20200930,235500,1.17237,1.17248,1.17180,1.17207,3880
EURUSD,5,20201001,0,1.17216,1.17235,1.17185,1.17190,4947


In [68]:
def divide_numdate(num_date):
    return f'{num_date[0:4]}:{num_date[4:6]}:{num_date[6:8]}'

def divide_timedate(time_date, pairs_count=3):
    time_date = (6 - len(time_date))*'0' + time_date
    time_date = ':'.join([time_date[2*i:2*(i+1)] for i in range(pairs_count)])
    return time_date

def to_datetime_index_date(index_date):
    return datetime.datetime.strptime(index_date, '%Y:%m:%d %H:%M:%S')
    

In [ ]:
def 



In [102]:
test_df['ymd'] = test_df.numdate.apply(divide_numdate)
test_df['hms'] = test_df.time.apply(divide_timedate)

In [107]:
test_df['index_date'] = test_df.ymd + ' ' + test_df.hms

In [112]:
test_df.index_date.apply(to_datetime_index_date)

EURUSD   2020-07-14 06:05:00
EURUSD   2020-08-21 20:25:00
EURUSD   2020-07-13 19:30:00
EURUSD   2020-08-25 02:05:00
EURUSD   2020-08-11 10:55:00
EURUSD   2020-07-29 07:25:00
EURUSD   2020-08-04 12:20:00
EURUSD   2020-09-02 11:25:00
EURUSD   2020-08-03 15:45:00
EURUSD   2020-08-27 08:35:00
Name: index_date, dtype: datetime64[ns]